In [8]:
import os
import psycopg2
import requests
import json
import datetime
from pytz import timezone
import pandas as pd
import requests

In [41]:
perp = requests.get('https://api-testnet.bybit.com/v2/public/tickers').json()
df_perp = pd.DataFrame(perp['result'])
# df_perp['delivery_fee_rate'] = df_perp['delivery_fee_rate'].str.replace('', '0')
# df_perp['predicted_delivery_price'] = df_perp['predicted_delivery_price'].str.replace('', '0')
# df_perp['delivery_time'] = df_perp['delivery_time'].str.replace('', '1900-01-01T00:00:00Z')
# df_perp['next_funding_time'] = df_perp['next_funding_time'].str.replace('', '1900-01-01T00:00:00Z')

# df_perp = df_perp.fillna({'delivery_fee_rate': '0', 'predicted_delivery_price': '0', 'delivery_time': '1900-01-01T00:00:00Z', 'next_funding_time': '1900-01-01T00:00:00Z'})

# df.fillna({'name': 'XXX', 'age': 20, 'ZZZ': 100})

In [42]:
df_perp.loc[df_perp['delivery_fee_rate'].str.len() == 0, 'delivery_fee_rate'] = 0
df_perp.loc[df_perp['predicted_delivery_price'].str.len() == 0, 'predicted_delivery_price'] = 0
df_perp.loc[df_perp['delivery_time'].str.len() == 0, 'delivery_time'] = '1900-01-01T00:00:00Z'
df_perp.loc[df_perp['next_funding_time'].str.len() == 0, 'next_funding_time'] = '1900-01-01T00:00:00Z'

In [43]:
host="rdsnske.c7sczatjnphy.ap-northeast-1.rds.amazonaws.com"
dbname="bot"
user="rdsuser"
password="rds%5678"
insert_base = "insert into bybit_api_futures values('{basetime}','{symbol}','{bid_price}','{ask_price}','{last_price}','{last_tick_direction}','{prev_price_24h}','{price_24h_pcnt}','{high_price_24h}','{low_price_24h}','{prev_price_1h}','{price_1h_pcnt}','{mark_price}','{index_price}','{open_interest}','{open_value}','{total_turnover}','{turnover_24h}','{total_volume}','{volume_24h}','{funding_rate}','{predicted_funding_rate}','{next_funding_time}','{countdown_hour}','{delivery_fee_rate}','{predicted_delivery_price}','{delivery_time}');"

In [44]:
# データをDBに登録
with psycopg2.connect(host=host, dbname=dbname, user=user, password=password) as conn:
    with conn.cursor() as cur:
        basetime = datetime.datetime.now()
        basetime = basetime.astimezone(timezone('UTC'))
        basetime = datetime.datetime.strftime(basetime, '%Y-%m-%dT%H:%M:00')
        for d in df_perp.itertuples():
            insert_sql = insert_base.format(basetime=basetime
                                            ,symbol=d.symbol
                                            ,bid_price=d.bid_price
                                            ,ask_price=d.ask_price
                                            ,last_price=d.last_price
                                            ,last_tick_direction=d.last_tick_direction
                                            ,prev_price_24h=d.prev_price_24h
                                            ,price_24h_pcnt=d.price_24h_pcnt
                                            ,high_price_24h=d.high_price_24h
                                            ,low_price_24h=d.low_price_24h
                                            ,prev_price_1h=d.prev_price_1h
                                            ,price_1h_pcnt=d.price_1h_pcnt
                                            ,mark_price=d.mark_price
                                            ,index_price=d.index_price
                                            ,open_interest=d.open_interest
                                            ,open_value=d.open_value
                                            ,total_turnover=d.total_turnover
                                            ,turnover_24h=d.turnover_24h
                                            ,total_volume=d.total_volume
                                            ,volume_24h=d.volume_24h
                                            ,funding_rate=d.funding_rate
                                            ,predicted_funding_rate=d.predicted_funding_rate
                                            ,next_funding_time=d.next_funding_time
                                            ,countdown_hour=d.countdown_hour
                                            ,delivery_fee_rate=d.delivery_fee_rate
                                            ,predicted_delivery_price=d.predicted_delivery_price
                                            ,delivery_time=d.delivery_time)
            cur.execute(insert_sql)
    conn.commit()

In [14]:
df_perp.to_csv('df_perp.csv')

In [15]:
df_perp

,symbol,bid_price,ask_price,last_price,last_tick_direction,prev_price_24h,price_24h_pcnt,high_price_24h,low_price_24h,prev_price_1h,...,turnover_24h,total_volume,volume_24h,funding_rate,predicted_funding_rate,next_funding_time,countdown_hour,delivery_fee_rate,predicted_delivery_price,delivery_time
0,ETHUSD,4355.75,4359.4,4359.45,PlusTick,4297.60,0.014391,4448.25,4215.95,4353.65,...,4501.55,1.329678e+11,19615331.0,0.0001,0.0001,2021-11-21T16:00:00Z,7,0,,
1,BTCUSD,58880.5,58882,58882.00,MinusTick,58597.00,0.004863,60000.00,57178.00,58924.00,...,3040.50,4.389653e+11,178837556.0,0.0001,0.0001,2021-11-21T16:00:00Z,7,0,,
2,EOSUSD,4.275,4.277,4.278,PlusTick,4.330,-0.012009,4.360,4.182,4.273,...,1073846.970,5.201497e+10,4611477.0,0.000413,0.000651,2021-11-21T16:00:00Z,7,0,,
3,DOTUSD,41.845,41.85,41.865,ZeroPlusTick,41.615,0.006007,42.585,39.855,41.475,...,115615.912,2.544584e+08,4782080.0,0.0001,0.0001,2021-11-21T16:00:00Z,7,0,,
4,XRPUSD,1.0754,1.0756,1.0754,MinusTick,1.0881,-0.011671,1.1055,1.0575,1.0745,...,2736337.9512,1.036289e+11,2972018.0,0.0001,0.0001,2021-11-21T16:00:00Z,7,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,SLPUSDT,0.06465,0.0647,0.06465,MinusTick,0.06536,-0.010862,0.06753,0.06291,0.06468,...,1708911.82,1.193566e+08,26162070.0,0.0001,0.0001,2021-11-21T16:00:00Z,7,,,
75,BTCUSDZ21,59818,59838.5,59845.50,ZeroPlusTick,59473.50,0.006254,60886.00,58255.00,59835.00,...,337.36,4.493230e+09,20176646.0,0,0,,0,0.0005,0.00,2021-12-31T08:00:00Z
76,BTCUSDH22,61286,61295.5,61300.50,ZeroPlusTick,60884.50,0.006832,62407.50,59634.00,61264.00,...,280.38,1.960362e+09,17141033.0,0,0,,0,0.0005,0.00,2022-03-25T08:00:00Z
77,ETHUSDZ21,4412.3,4412.35,4412.30,ZeroMinusTick,4350.55,0.014193,4505.25,4261.15,4410.35,...,4108.66,2.806464e+09,18104268.0,0,0,,0,0.0005,0.00,2021-12-31T08:00:00Z


In [7]:
min(df_perp['time'])

'2021-11-18T14:53:35.000Z'